In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import os.path as op
import pandas as pd
import json
import base64

In [3]:
sys.path.append(op.abspath('..'))

In [4]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

In [5]:
import torch
import multiprocessing
from itertools import chain
import numpy as np
import random

In [6]:
from bokeh.io import output_notebook, output_file
from bokeh.plotting import figure, show
from bokeh.models import LinearColorMapper, ColumnDataSource
from bokeh.transform import factor_cmap, factor_mark
from torch.utils.data import DataLoader


from datasets import SLREmbeddingDataset, collate_fn_padd
from datasets.dataset_loader import LocalDatasetLoader
from models import embeddings_scatter_plot_splits
from models import SPOTER_EMBEDDINGS

In [7]:
seed = 43
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True) 
generator = torch.Generator()
generator.manual_seed(seed)

In [8]:
BASE_DATA_FOLDER = '../data/'
os.environ["BASE_DATA_FOLDER"] = BASE_DATA_FOLDER
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [9]:
# LOAD MODEL FROM CLEARML
# from clearml import InputModel
# model = InputModel(model_id='1b736da469b04e91b8451d2342aef6ce')
# checkpoint = torch.load(model.get_weights())


CHECKPOINT_PATH = "../checkpoints/checkpoint_embed_992.pth"
checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)


model = SPOTER_EMBEDDINGS(
    features=checkpoint["config_args"].vector_length,
    hidden_dim=checkpoint["config_args"].hidden_dim,
    norm_emb=checkpoint["config_args"].normalize_embeddings,
).to(device)

model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [10]:
SL_DATASET = 'wlasl'  # or 'lsa'
if SL_DATASET == 'wlasl':
    dataset_name = "wlasl_mapped_mediapipe_only_landmarks_25fps"
    num_classes = 100
    split_dataset_path = "WLASL100_{}_25fps.csv"
else:
    dataset_name = "lsa64_mapped_mediapipe_only_landmarks_25fps"
    num_classes = 64
    split_dataset_path = "LSA64_{}.csv"
    

In [11]:
def get_dataset_loader(loader_name=None):
    if loader_name == 'CLEARML':
        from datasets.clearml_dataset_loader import ClearMLDatasetLoader
        return ClearMLDatasetLoader()
    else:
        return LocalDatasetLoader()

dataset_loader = get_dataset_loader()
dataset_project = "Sign Language Recognition"
batch_size = 1
dataset_folder = dataset_loader.get_dataset_folder(dataset_project, dataset_name)

In [12]:
dataloaders = {}
splits = ['train', 'val']
dfs = {}
for split in splits:
    split_set_path = op.join(dataset_folder, split_dataset_path.format(split))
    split_set = SLREmbeddingDataset(split_set_path, triplet=False)
    data_loader = DataLoader(
        split_set,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn_padd,
        pin_memory=torch.cuda.is_available(),
        num_workers=multiprocessing.cpu_count()
    )
    dataloaders[split] = data_loader
    dfs[split] =  pd.read_csv(split_set_path)

with open(op.join(dataset_folder, 'id_to_label.json')) as fid:
    id_to_label = json.load(fid)
id_to_label = {int(key): value for key, value in id_to_label.items()}

tsne_results, labels_results = embeddings_scatter_plot_splits(model,
                                                              dataloaders,
                                                              device,
                                                              id_to_label,
                                                              perplexity=40,
                                                              n_iter=1000)


set_labels = list(set(next(chain(labels_results.values()))))

/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [18]:
dfs = {}
for split in splits:
    split_set_path = op.join(dataset_folder, split_dataset_path.format(split))
    df =  pd.read_csv(split_set_path)
    df['tsne_x'] = tsne_results[split][:, 0]
    df['tsne_y'] = tsne_results[split][:, 1]
    df['split'] = split
    if SL_DATASET == 'wlasl':
        df['video_fn'] = df['video_id'].apply(lambda video_id: os.path.join(BASE_DATA_FOLDER, f'wlasl/videos/{video_id:05d}.mp4'))
    else:
        df['video_fn'] = df['video_id'].apply(lambda video_id: os.path.join(BASE_DATA_FOLDER, f'lsa/videos/{video_id}.mp4'))
    dfs[split] = df

df = pd.concat([dfs['train'].sample(100), dfs['val']]).reset_index(drop=True)
len(df)

1533

In [19]:
from tqdm.auto import tqdm

def load_videos(video_list):
    print('loading videos')
    videos = []
    for video_fn in tqdm(video_list):
        if video_fn is None:
            video_data = None
        else:
            with open(video_fn, 'rb') as fid:
                video_data = base64.b64encode(fid.read()).decode()
        videos.append(video_data)
    print('Done loading videos')
    return videos

In [20]:
use_img_div = False
if use_img_div:
    # sample dataframe data to avoid overloading scatter plot with too many videos
    df = df.loc[(df['tsne_x'] > 10) & (df['tsne_x'] < 20)]
    df = df.loc[(df['tsne_y'] > 10) & (df['tsne_y'] < 20)]

In [21]:
img_div = '''
        <div>
            <video autoplay src="data:video/mp4;base64,@videos" height="90" width="120">
            </video>
        </div>
'''
TOOLTIPS = f"""
    <div>
        {img_div if use_img_div else ''}
        <div>
            <span style="font-size: 17px; font-weight: bold;">@label_desc - @split</span>
            <span style="font-size: 15px; color: #966;">[#@video_id]</span>
        </div>
        </div>
    </div>
"""
cmap = LinearColorMapper(palette="Turbo256", low=0, high=len(set_labels))

output_notebook()
# or 
# output_file("scatter_plot.html")

p = figure(width=1000,
           height=800,
           tooltips=TOOLTIPS,
           title=f"Check {'video' if use_img_div else 'label'} by hovering mouse over the dots")

Loading BokehJS ...

In [22]:
column_data = dict(
    x=df['tsne_x'],
    y=df['tsne_y'],
    label=df['labels'],
    label_desc=df['label_name'],
    split=df['split'],
    video_id=df['video_id']
)

if use_img_div:
    emb_videos = load_videos(df['video_fn'])
    column_data["videos"] = emb_videos
source = ColumnDataSource(data=column_data)

p.scatter('x', 'y',
     size=10,
     source=source,
     fill_color={"field": 'label', "transform": cmap},
     line_color={"field": 'label', "transform": cmap},         
     #legend_label={"field": 'split', "transform": lambda x: df['split']},
#      marker={"field": 'split'}
         )

show(p)

In [ ]:
df